## SVM phiên bản cũ

Phiên bản này dùng linear kernel và tối ưu bằng thuật toán gradient descent.

Linear kernel: w * x + b 

Ý tưởng của phiên bản này gồm tính toán gradient của hai tham số w, b theo từng batch, sau đó dùng gradient này để cập nhật lại w và b.

In [ ]:
def fit(self, X, Y, batch_size=100, learning_rate=0.001, epochs=1000):
        # số thuộc tính
        number_of_features = X.shape[1]

        # số mẫu
        number_of_samples = X.shape[0]

        c = self.C

        # mảng index để truy xuất dữ liệu trong từng batch
        ids = np.arange(number_of_samples)

        # hoán vị các mẫu
        np.random.shuffle(ids)

        # khởi tạo các giá trị w,b và losses
        w = np.zeros((1, number_of_features))
        b = 0
        losses = []

        # Thuật toán gradient descent
        for i in range(epochs):
            # tính toán hàm mất mát
            l = self.hingeloss(w, b, X, Y)
            losses.append(l)
            
            #tính toán gradient của w và b trong từng batch
            for batch_initial in range(0, number_of_samples, batch_size):
                gradw = 0
                gradb = 0

                # Với mỗi phần tử trong batch tính:
                # ti = y * ((w @ x) + b)
                # nếu ti <= 1 thì gradient được cập nhật như sau:
                # gradw = gradw + c * y * x
                # gradb = gradb + c * y
                # sau khi đã tính toán xong gradient trong batch thì w và b được cập nhật
                # w = w - learning_rate * w + learning_rate * gradw
                # b = b + learning_rate * gradb
                for j in range(batch_initial, batch_initial + batch_size):
                    if j < number_of_samples:
                        x = ids[j]
                        ti = Y[x] * (np.dot(w, X[x].T) + b)

                        if ti > 1:
                            gradw += 0
                            gradb += 0
                        else:
                            # tính toán gradient
                            gradw += c * Y[x] * X[x]
                            gradb += c * Y[x]

                # Cập nhật w và b
                w = w - learning_rate * w + learning_rate * gradw
                b = b + learning_rate * gradb
        
        self.w = w
        self.b = b

        return self.w, self.b, losses

## SVM phiên bản tuần tự

### Phương pháp phân rã

Phương pháp này điều chỉnh tập con của alpha trong mỗi vòng lặp, do đó chỉ vài cột Q là cần thiết (Q là một ma trận dày - dense và có thể rất lớn để lưu trữ). Tập con của những biến này thường được gọi là tập làm việc (working set) B, dẫn đến một vấn đề tối ưu phụ nhỏ hơn.

Bởi vì chỉ có một vài thành phần được cập nhật trong mỗi vòng lặp, với những bài toán khó, phương pháp phân rã này có sự hội tụ chậm. Những phương pháp tốt hơn cho việc lựa chọn tập làm việc có thể giảm số vòng lặp xuống.

### Hàm tính Q

Q là ma trận đối xứng với từng phần tử được tính theo công thức $Q_{ij} = y_iy_jK(x_i, x_j)$ với $K(x_i, x_j)$ là hàm kernel.

```python
def get_Q(self, X, i, j):
  if self.Q[i, j] == None:
    self.Q[i][j] = self.y[i] * self.y[j] * self.get_K(X, i, j)
    self.Q[j][i] = self.Q[i][j]
  return self.Q[i][j] 
```

### Hàm lựa chọn bộ làm việc (working set selection - WSS)

**WSS 3**

**(Working set selection using second order information: any symmetric $K$)**

1. Định nghĩa $a_{ts}$ và $b_{ts}$ như sau:

$a_{ts} \equiv K_{tt} + K_{ss} - 2K_{ts} > 0$ và $b_{ts} \equiv -y_t \nabla f(\alpha^k)_t + y_s \nabla f(\alpha^k)_s > 0$

và

$\bar a \equiv \Biggl\{ {a_{ts} \; nếu \; a_{ts} > 0, \\ \tau \; khác.}$

Chọn

$i \in arg max_t \{-y_t \nabla f(\alpha^k)_t | t \in I_{up}(\alpha^k)\}$

$j \in argmin_t \{ -\frac{b^2_{it}}{\bar a_{it}} | t \in I_{low}(\alpha^k), -y_t \nabla f(\alpha^k)_t < -y_i \nabla f(\alpha^k)_i \}  $

2. Trả về $B = \{i, j\}$

Đây là một phương pháp trong việc phân rã để huấn luyện thuật toán SVM. Thuật toán này dùng đạo hàm bậc hai cho bất kỳ ma trận K đối xứng nào. Hàm này giải quyết được vấn đề chứng minh sự hội tụ của phương pháp phân rã. Đồng thời thuật toán này cũng giảm số vòng lặp.

```python
def select_B(self, X):
  i = -1
  G_max = -np.inf
  G_min = np.inf
  for t in range(self.l):
    if (self.y[t] == 1 and self.alphas[t] <self.C) or \
    (self.y[t] == -1 and self.alphas[t] > 0):
      if -self.y[t] * self.G[t] >= G_max:
        i = t
        G_max = -self.y[t] * self.G[t]
        
  j = -1
  obj_min = np.inf
  for t in range(self.l):
    if (self.y[t]==1 and self.alphas[t]>0) or \
        (self.y[t] == -1 and self.alphas[t] < self.C):
      b = G_max + self.y[t] * self.G[t]
      if -self.y[t]*self.G[t] <= G_min:
        G_min = -self.y[t] * self.G[t]
      if b>0:
        a = self.get_Q(X, i, i) + self.get_Q(X, t, t) - 2.0*self.y[i]*self.y[t]*self.get_Q(X, i, t)
        if a<=0:
          a = self.tau
        if -(b*b)/a <= obj_min:
          j = t
          obj_min = -(b*b)/a
  if G_max - G_min < self.eps:
    return -1, -1
  return i, j
```

Hàm này còn bao gồm công dụng kiểm tra điều kiện dừng của thuật toán `G_max - G_min < eps`

#### Cập nhật Gradient

$\nabla f(\alpha^{k+1}) = \nabla f(\alpha^k) + Q_{:,B}(\alpha^{k+1}-\alpha^k)$

```python
for t in range(self.l):
                self.G[t] += self.get_Q(X, i, t) * delta_ai + self.get_Q(X, j, t) *delta_aj
```

### Tìm threshold b:

Ta có công thức: $-b=\frac{\sum_{i:0<\alpha_i<C>}y_i\nabla_i f (\alpha))}{|\{i|0<\alpha_i<C>\}|}$

Nếu không có $\alpha$ nào thỏa $0<\alpha_i<C$ thì: điều kiện KKT chuyển thành:

$-M(\alpha) = max\{y_i\nabla_i f(\alpha) | \alpha_i=0,y_i=-1 \ or\  \alpha_i=C,y_i=1\}
<=-b<=-m(\alpha) = min\{y_i\nabla_i f(\alpha) | \alpha_i=0,y_i=1 \ or\  \alpha_i=C,y_i=-1\}$

Ta sẽ chọn -b ở khoảng giữa

```python
    def get_b(self):
        sum = 0.0
        count = 0
        for i in range(self.l):
            if 0 < self.alphas[i] < self.C:
                count += 1
                sum += self.y[i] * self.G[i]
        if count > 0:
            self.b = sum/count
            return
        max = -np.inf
        min = np.inf
        for i in range(self.l):
            if (self.alphas[i] == 0 and self.y[i] == -1) or \
                (self.alphas[i] == self.C and self.y[i] == 1):
                    if max < self.y[i] * self.G[i]:
                        max = self.y[i] * self.G[i]
            if (self.alphas[i] == 0 and self.y[i] == 1) or \
                (self.alphas[i] == self.C and self.y[i] == -1):
                    if min > self.y[i] *self.G[i]:
                        min = self.y[i] * self.G[i]
        self.b = (min+max) / 2
```

### Code hoàn chỉnh

In [ ]:
class SVM_Se:
    # Khởi tạo các tham số
    def __init__(self, gamma = -1, kernel = 'rbf', C = 1.0, eps = 1e-3):
        self.C = C
        self.eps = eps # điều kiện dừng
        self.gamma = gamma
        self.tau = 1e-12
        self.kernel = kernel
        self.gamma = gamma
    
    # hàm kernel
    def Kernel(self, x1, x2):
        # rbf kernel
        if self.kernel == 'rbf':
            return self.rbf(x1, x2)
        # linear kernel
        if self.kernel == 'linear':
            return self.linear(x1, x2)

    # hàm tính ma trận Q
    def get_Q(self, X, i, j):
        if self.Q[i, j] == None:
            self.Q[i][j] = self.y[i] * self.y[j] * self.get_K(X, i, j)
            self.Q[j][i] = self.Q[i][j]
        return self.Q[i][j]
    
    # hàm tính giá trị kernel
    def get_K(self, X, i, j):
        if self.K[i, j] == None:
            self.K[i][j] = self.Kernel(X[i], X[j])
            self.K[j][i] = self.K[i][j]
        return self.K[i][j]

    # hàm tính linear kernel là tích vô hướng của hai vecto
    def linear(self, x1, x2):
        x1_temp = x1.astype(np.float64)
        x2_temp = x2.astype(np.float64)
        return x1_temp.dot(x2_temp)
    
    # hàm tính rbf kernel theo công thức: e^(-gamma *(x1 @ x1 + x2 @ x2 - 2 * (x1 @ x2)))
    def rbf(self, x1, x2):
        x1_temp = x1.astype(np.float64)
        x2_temp = x2.astype(np.float64)
        return np.exp(-self.gamma * (x1_temp.dot(x1_temp) + x2_temp.dot(x2_temp) - 2.0 * x1_temp.dot(x2_temp)))
        
    # hàm lựa chọn bộ làm việc
    def select_B(self, X):
        i = -1
        G_max = -np.inf
        G_min = np.inf
        for t in range(self.l):
            if (self.y[t] == 1 and self.alphas[t] <self.C) or \
            (self.y[t] == -1 and self.alphas[t] > 0):
                if -self.y[t] * self.G[t] >= G_max:
                    i = t
                    G_max = -self.y[t] * self.G[t]
        j = -1
        obj_min = np.inf
        for t in range(self.l):
            if (self.y[t]==1 and self.alphas[t]>0) or \
                (self.y[t] == -1 and self.alphas[t] < self.C):
                    b = G_max + self.y[t] * self.G[t]
                    if -self.y[t]*self.G[t] <= G_min:
                        G_min = -self.y[t] * self.G[t]
                    if b>0:
                        a = self.get_Q(X, i, i) + self.get_Q(X, t, t) - 2.0*self.y[i]*self.y[t]*self.get_Q(X, i, t)
                        if a<=0:
                            a = self.tau
                        if -(b*b)/a <= obj_min:
                            j = t
                            obj_min = -(b*b)/a
        if G_max - G_min < self.eps:
            return -1, -1
        return i, j

    # hàm dự đoán
    def predict(self, X):
        pred = []
        for x in X:
            sum = 0.0
            for i in range(self.l):
                sum += self.y[i] * self.alphas[i] * self.Kernel(self.X[i], x)
            sum -= self.b
            pred.append(np.sign(sum))
        return pred


    def get_b(self):
        sum = 0.0
        count = 0
        for i in range(self.l):
            if 0 < self.alphas[i] < self.C:
                count += 1
                sum += self.y[i] * self.G[i]
        if count > 0:
            self.b = sum/count
            return
        max = -np.inf
        min = np.inf
        for i in range(self.l):
            if (self.alphas[i] == 0 and self.y[i] == -1) or \
                (self.alphas[i] == self.C and self.y[i] == 1):
                    if max < self.y[i] * self.G[i]:
                        max = self.y[i] * self.G[i]
            if (self.alphas[i] == 0 and self.y[i] == 1) or \
                (self.alphas[i] == self.C and self.y[i] == -1):
                    if min > self.y[i] *self.G[i]:
                        min = self.y[i] * self.G[i]
        self.b = (min+max) / 2

    # hàm huấn luyện mô hình
    def fit(self, X, y):
        global t1
        global t2
        global t3
        self.y = y
        self.X = X
        self.l = len(y)
        if self.gamma == -1:
            self.gamma = 1/(X.shape[1]*X.var())
        self.active_size= self.l

        # khởi tạo mảng alpha
        self.alphas = np.zeros(self.l)
        self.n_iter = 0

        self.K = np.array([[None for _ in range(self.l)] for _ in range(self.l)])
        self.Q = np.array([[None for _ in range(self.l)] for _ in range(self.l)])
        
        # khởi tạo mảng gradient gồm các số -1
        self.G = np.array([-1.0 for _ in range(self.l)])
        
        while True:
            i, j = self.select_B(X)
            if j == -1:
                break
            self.n_iter += 1
            alphai = self.alphas[i]
            alphaj = self.alphas[j]
            if y[i] != y[j]:
                quad_coef = self.get_Q(X, i, i) + self.get_Q(X, j, j) + 2*self.get_Q(X, i, j)
                if quad_coef <= 0:
                    quad_coef = self.tau
                delta = (-self.G[i] - self.G[j])/quad_coef
                diff = alphai - alphaj
                self.alphas[i] += delta
                self.alphas[j] += delta
                if diff > 0:
                    if self.alphas[j]<0:
                        self.alphas[j] = 0
                        self.alphas[i] = diff
                    if self.alphas[i]>self.C:
                        self.alphas[i] = self.C
                        self.alphas[j] = self.C - diff
                else:
                    if self.alphas[i]<0:
                        self.alphas[i] = 0
                        self.alphas[j] = -diff
                    if self.alphas[j]>self.C:
                        self.alphas[j] = self.C
                        self.alphas[i] = self.C + diff
            else:
                quad_coef = self.get_Q(X, i, i) + self.get_Q(X,j,j) - 2*self.get_Q(X,i,j)
                if quad_coef <=0:
                    quad_coef = self.tau
                delta = (self.G[i]-self.G[j])/quad_coef
                sum = alphai + alphaj
                self.alphas[i] -= delta
                self.alphas[j] += delta
                if sum>self.C:
                    if self.alphas[i]>self.C:
                        self.alphas[i] = self.C
                        self.alphas[j] = sum-self.C
                    if self.alphas[j]>self.C:
                        self.alphas[j] = self.C
                        self.alphas[i] = sum-self.C
                else:
                    if self.alphas[j]<0:
                        self.alphas[j] = 0
                        self.alphas[i] = sum
                    if self.alphas[i]<0:
                        self.alphas[i] = 0
                        self.alphas[j] = sum
            delta_ai = self.alphas[i] - alphai
            delta_aj = self.alphas[j] - alphaj
            for t in range(self.l):
                self.G[t] += self.get_Q(X, i, t) * delta_ai + self.get_Q(X, j, t) *delta_aj
        self.get_b()

## SVM phiên bản song song

**1. Khó khăn:**
- SMO (Sequential Minimal Optimization) bản chất là thuật toán tuần tự.
Hầu như không phân thành các thao tác độc lập để song song.

**2. Giải quyết:**
- Phân thuật toán thành 3 giai đoạn lớn và đo tổng thời gian chạy: xác định cặp điểm cần tối ưu, tối ưu cặp điểm, cập nhật Gradient.

- Thuật toán cần gọi hàm tính kernel $K(i, j)$ và tính giá trị $Q_{ij} = y_i y_j K(i, j)$ rất nhiều lần → Tính tổng thời gian mà thao tác này chiếm.

**3. Song song hóa thuật toán SVM mới:**

Thời gian tính $Q$ chiếm đa số thời lượng fit, dù thao tác tính $Q_{ij}$ không mất nhiều thời gian nhưng lại được gọi rất nhiều lần.

→ Tìm toàn bộ $Q$ bằng song song và lưu vào 1 matrix. Khi cần chỉ cần lấy ra.

Hàm Predict tốn nhiều thời gian bởi phải tính giá trị $K_{ij}$ nhiều lần (tương tự như tính $Q$ ở fit). → Giải quyết tương tự.

Sau khi tính $Q$ toàn bộ, có thể cập nhật Gradient nhanh hơn bằng các thao tác với ma trận thay vì vòng for.

Ngoài ra còn có thể tính dual_coef = y * alphas bằng song song thay vì lưu y và alpha để tối ưu quá trình fit.

**4. Đánh giá**
- Ưu điểm:

Tối ưu hóa hiệu quả tổng thời gian fit và predict.

Không có nguy cơ mất mát thông tin như cơ chế shrink của sklearn (cơ chế giúp tăng tốc khi lượng data lớn) nhưng vẫn có tốc độ cao hơn sklearn.

- Nhược điểm:

Tốn nhiều VRAM vì phải load toàn bộ data lên GPU để tìm $Q$.

Không xác định được kích thước block tối ưu vì dữ liệu có thể ít nhiều khác nhau.


In [ ]:
class SVM_Pa:
    def __init__(self, gamma = -1, kernel = 'rbf', C = 1.0, eps = 1e-3):
        self.C = C
        self.eps = eps
        self.gamma = gamma
        self.tau = 1e-12
        self.kernel = kernel
        self.gamma = gamma
    
    def Kernel(self, x1, x2):
        if self.kernel == 'rbf':
            return self.rbf(x1, x2)
        if self.kernel == 'linear':
            return self.linear(x1, x2)
   
    def linear(self, x1, x2):
        x1_temp = x1.astype(np.float64)
        x2_temp = x2.astype(np.float64)
        return x1_temp.dot(x2_temp)
    
    def rbf(self, x1, x2):
        x1_temp = x1.astype(np.float64)
        x2_temp = x2.astype(np.float64)
        return np.exp(-self.gamma * (x1_temp.dot(x1_temp) + x2_temp.dot(x2_temp) - 2.0 * x1_temp.dot(x2_temp)))
        
    def select_B(self, X):
        i = -1
        G_max = -np.inf
        G_min = np.inf
        for t in range(self.l):
            if (self.y[t] == 1 and self.alphas[t] <self.C) or \
            (self.y[t] == -1 and self.alphas[t] > 0):
                if -self.y[t] * self.G[t] >= G_max:
                    i = t
                    G_max = -self.y[t] * self.G[t]
        j = -1
        obj_min = np.inf
        for t in range(self.l):
            if (self.y[t]==1 and self.alphas[t]>0) or \
                (self.y[t] == -1 and self.alphas[t] < self.C):
                    b = G_max + self.y[t] * self.G[t]
                    if -self.y[t]*self.G[t] <= G_min:
                        G_min = -self.y[t] * self.G[t]
                    if b>0:
                        a = self.Q[i, i] + self.Q[t, t] - 2.0*self.y[i]*self.y[t]*self.Q[i, t]
                        if a<=0:
                            a = self.tau
                        if -(b*b)/a <= obj_min:
                            j = t
                            obj_min = -(b*b)/a
        if G_max - G_min < self.eps:
            return -1, -1
        return i, j

    def predict(self, X):
        return np.sign(self.dual_coef.dot(self.init_K(X)) - self.b)

    @staticmethod
    @cuda.jit 
    def init_K_kernel_rbf(X1, X2, K, n1, n2, m, gamma):
        i, j = cuda.grid(2)
        if i>= n1 or j >= n2:
            return
        sumii = np.float64(0)
        sumij = np.float64(0)
        sumjj = np.float64(0)
        for k in range(m):
            sumii += X1[i][k] * X1[i][k]
            sumij += X1[i][k] * X2[j][k]
            sumjj += X2[j][k] * X2[j][k]
        K[i, j] = math.exp(-gamma * (sumii + sumjj - 2.0 * sumij))
    
    @staticmethod
    @cuda.jit 
    def init_K_kernel_linear(X1, X2, K, n1, n2, m):
        i, j = cuda.grid(2)
        if i>= n1 or j >= n2:
            return
        sumij = np.float64(0)
        for k in range(m):
            sumij += X1[i][k] * X2[j][k]
        K[i, j] = sumij
        
    def init_K(self, x):
        d_x1 = self.X.astype(np.float64)
        d_x2 = x.astype(np.float64)
        d_K = cuda.device_array((self.l, x.shape[0]), np.float64)
        blocksize = (32, 32)
        gridsize = (math.ceil(self.l/blocksize[0]), math.ceil(x.shape[0]/blocksize[1]))
        if self.kernel == 'rbf':
            self.init_K_kernel_rbf[gridsize, blocksize](d_x1, d_x2, d_K, self.l, x.shape[0], self.n_features, self.gamma)
        elif self.kernel == 'linear':
            self.init_K_kernel_linear[gridsize, blocksize](d_x1, d_x2, d_K, self.l, x.shape[0], self.n_features)
        return np.array(d_K.copy_to_host())

    def get_b(self):
        sum = 0.0
        count = 0
        for i in range(self.l):
            if 0 < self.alphas[i] < self.C:
                count += 1
                sum += self.y[i] * self.G[i]
        if count > 0:
            self.b = sum/count
            return
        max = -np.inf
        min = np.inf
        for i in range(self.l):
            if (self.alphas[i] == 0 and self.y[i] == -1) or \
                (self.alphas[i] == self.C and self.y[i] == 1):
                    if max < self.y[i] * self.G[i]:
                        max = self.y[i] * self.G[i]
            if (self.alphas[i] == 0 and self.y[i] == 1) or \
                (self.alphas[i] == self.C and self.y[i] == -1):
                    if min > self.y[i] *self.G[i]:
                        min = self.y[i] * self.G[i]
        self.b = (min+max) / 2
    
    @staticmethod
    @cuda.jit 
    def init_Q_kernel_rbf(X, y, Q, n, m, gamma):
        i, j = cuda.grid(2)
        if i>= n or j >= n:
            return
        sumii = np.float64(0)
        sumij = np.float64(0)
        sumjj = np.float64(0)
        for k in range(m):
            sumii += X[i][k] * X[i][k]
            sumij += X[i][k] * X[j][k]
            sumjj += X[j][k] * X[j][k]
        Q[i, j] = y[i]*y[j]*math.exp(-gamma * (sumii + sumjj - 2.0 * sumij))
    
    
    @staticmethod
    @cuda.jit 
    def init_Q_kernel_linear(X, y, Q, n, m):
        i, j = cuda.grid(2)
        if i>= n or j >= n:
            return
        sumij = np.float64(0)
        for k in range(m):
            sumij += X[i][k] * X[j][k]
        Q[i, j] = y[i]*y[j]*sumij
        
    def init_Q(self):
        d_x = self.X.astype(np.float64)
        d_y = self.y.astype(np.float64)
        d_Q = cuda.device_array((self.l, self.l), np.float64)
        blocksize = (32, 32)
        gridsize = (math.ceil(self.l/blocksize[0]), math.ceil(self.l/blocksize[1]))
        if self.kernel == 'rbf':
            self.init_Q_kernel_rbf[gridsize, blocksize](d_x, d_y, d_Q, self.l, self.n_features, self.gamma)
        elif self.kernel == 'linear':
            self.init_Q_kernel_linear[gridsize, blocksize](d_x, d_y, d_Q, self.l, self.n_features)
        self.Q = np.array(d_Q.copy_to_host())
    
    @staticmethod
    @cuda.jit
    def compute_dual_coef_kernel(alpha, y, dual_coef, l):
        i = cuda.grid(1)
        if i > l:
            return
        dual_coef[i] = alpha[i] * y[i]

    def compute_dual_coef(self):
        blocksize = 32
        gridsize = math.ceil(self.l/blocksize)
        d_dualcoef = cuda.device_array(self.l, np.float64)
        self.compute_dual_coef_kernel[gridsize, blocksize](self.alphas, self.y, d_dualcoef, self.l)
        self.dual_coef = d_dualcoef.copy_to_host()
        

    def fit(self, X, y):
        self.y = y
        self.X = X
        self.l, self.n_features = X.shape
        if self.gamma == -1:
            self.gamma = 1/(self.n_features*X.var())
        self.alphas = np.zeros(self.l)
        self.n_iter = 0
        self.init_Q()
        self.G = np.array([-1.0 for _ in range(self.l)])
        while True:
            i, j = self.select_B(X)
            if j == -1:
                break
            self.n_iter += 1
            alphai = self.alphas[i]
            alphaj = self.alphas[j]
            if y[i] != y[j]:
                quad_coef = self.Q[i, i] + self.Q[j, j] + 2*self.Q[i, j]
                if quad_coef <= 0:
                    quad_coef = self.tau
                delta = (-self.G[i] - self.G[j])/quad_coef
                diff = alphai - alphaj
                self.alphas[i] += delta
                self.alphas[j] += delta
                if diff > 0:
                    if self.alphas[j]<0:
                        self.alphas[j] = 0
                        self.alphas[i] = diff
                    if self.alphas[i]>self.C:
                        self.alphas[i] = self.C
                        self.alphas[j] = self.C - diff
                else:
                    if self.alphas[i]<0:
                        self.alphas[i] = 0
                        self.alphas[j] = -diff
                    if self.alphas[j]>self.C:
                        self.alphas[j] = self.C
                        self.alphas[i] = self.C + diff
            else:
                quad_coef = self.Q[i, i] + self.Q[j,j] - 2*self.Q[i,j]
                if quad_coef <=0:
                    quad_coef = self.tau
                delta = (self.G[i]-self.G[j])/quad_coef
                sum = alphai + alphaj
                self.alphas[i] -= delta
                self.alphas[j] += delta
                if sum>self.C:
                    if self.alphas[i]>self.C:
                        self.alphas[i] = self.C
                        self.alphas[j] = sum-self.C
                    if self.alphas[j]>self.C:
                        self.alphas[j] = self.C
                        self.alphas[i] = sum-self.C
                else:
                    if self.alphas[j]<0:
                        self.alphas[j] = 0
                        self.alphas[i] = sum
                    if self.alphas[i]<0:
                        self.alphas[i] = 0
                        self.alphas[j] = sum
            delta_ai = self.alphas[i] - alphai
            delta_aj = self.alphas[j] - alphaj
            self.G += self.Q[i, :] *delta_ai + self.Q[j, :]*delta_aj
        self.compute_dual_coef()
        self.get_b()

# Tài liệu tham khảo

SVM bản cũ:
https://www.pycodemates.com/2022/10/implementing-SVM-from-scratch-in-python.html

SVM bản mới:
http://cs229.stanford.edu/materials/smo.pdf

https://www.microsoft.com/en-us/research/publication/sequential-minimal-optimization-a-fast-algorithm-for-training-support-vector-machines/

https://www.csie.ntu.edu.tw/~cjlin/libsvm/

https://www.csie.ntu.edu.tw/~cjlin/papers/libsvm.pdf

https://www.csie.ntu.edu.tw/~cjlin/papers/quadworkset.pdf

